In [ ]:
#-*- coding: utf-8 -*-

<img align="right" style="max-width: 200px; height: auto" src="cfds_logo.png">

###  Lab 08 - "Supervised Deep Learning - Long Short-Term Memory Neural Networks"

Chartered Financial Data Scientist (CFDS), Autumn Term 2019/20

In the last lab you learned about how to utilize an **unsupervised** (deep) learning technique namely **Convolutional Neural Networks (CNNs)** to classify tiny images of objects contained in the CIFAR-10 dataset.

In this seventh lab, we will learn how to apply another type of deep learning technique referred to as **Long-Short-Term-Memory (LSTM)** neural networks. Unlike standard feedforward neural networks, LSTMs encompass feedback connections that make it a "general-purpose computer". LSTMs are designed to process not only a single data point (such as images), but also entire sequences of data, e.g., such as speech, video, or financial time series.


We will again use the functionality of the **'PyTorch'** library to implement and train an LSTM based neural network. The network will be trained on the historic daily (in-sample) returns of an exemplary financial stock. Once the network is trained, we will use the learned model to predict future (out-of-sample) returns. Finally, we will convert the predictions into tradable signals and the backtest the signals accordingly. 

The figure below illustrates a high-level eye view on the machine learning process we aim to establish in this lab.

<img align="center" style="max-width: 700px" src="process.png">

As always, pls. don't hesitate to ask all your questions either during the lab or send us an email via marco.schreyer@fds.ai or damian.borth@fds.ai.

### Lab Objectives:

After today's lab, you should be able to:

> 1. Understand the basic concepts, intuitions and major building blocks of **Long-Short Term Memory (LSTM) Neural Networks**.
> 2. Know how to **implement and to train an LSTM** to learn a model of financial time-series data.
> 3. Understand how to apply such a learned model to **predict future data points of a time-series**.
> 4. Know how to **interpret the model's prediction results** and backtest the predictions. 

### Setup of the Jupyter Notebook Environment

Prior to importing the needed lab libraries, let's install the `quandl`, `bt`, and `urllib` library which are not available in the off the shelf CoLab instance. This can be accomplished, as always, by using the Python packacke installer `pip`:

In [ ]:
!pip install quandl
!pip install bt
!pip install urllib

Similar to the previous labs, we need to import a couple of Python libraries that allow for data analysis and data visualization. We will mostly use the PyTorch, Numpy, Sklearn, Matplotlib, Seaborn, BT and a few utility libraries throughout the lab:

In [ ]:
# import python data science libraries
import os, io, urllib
import datetime as dt
import pandas as pd
import numpy as np

Import financial data science libraries:

In [ ]:
import quandl as ql # library to retreive financial data
import bt as bt # library to backtest trading signals

Import Python machine / deep learning libraries:

In [ ]:
# pytorch libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from torch.utils.data import dataloader

Import the sklearn pre-processing functions:

In [ ]:
# sklearn libraries
from sklearn import preprocessing

Import Python plotting libraries and set general plotting parameters:

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi']= 150

Enable notebook matplotlib inline plotting:

In [ ]:
%matplotlib inline

Suppress potential warnings:

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Create notebook folder structure to store the data as well as the trained neural network models:

In [ ]:
if not os.path.exists('./data'): os.makedirs('./data')  # create data directory
if not os.path.exists('./models'): os.makedirs('./models')  # create trained models directory

Set random seed value to obtain reproducable results:

In [ ]:
# init deterministic seed
seed_value = 1234
np.random.seed(seed_value) # set numpy seed
torch.manual_seed(seed_value) # set pytorch seed CPU

Enable GPU computing by setting the `use_gpu` flag and init a `CUDA` seed:

In [ ]:
# set gpu enabled flag
use_gpu = True

# init deterministic GPU seed
torch.cuda.manual_seed(seed_value)

Let's double check if we have access to a GPU provided by Google's CoLab environment:

In [ ]:
!nvidia-smi

### 1. Dataset Download and Data Assessment

In this section of the lab notebook we will download and access historic daily stock market data ranging from **01/01/2000** to **31/12/2017** of the **"International Business Machines" (IBM)** corporation (ticker symbol: "IBM"). To start the data download, let's initialize the "quandl" financial data download API and set an API key:

In [ ]:
#ql.ApiConfig.api_key = '<enter you own quandl api code here>'
ql.ApiConfig.api_key = 'xn6g-K_ebmMgSJRTCSUk'

Furthermore, let's specify the start and end date of the stock market data download:

In [ ]:
start_date = dt.datetime(2000, 1, 1)
end_date = dt.datetime(2017, 12, 31)

Download the daily "International Business Machines" (IBM) stock market data:

In [ ]:
stock_data = ql.get('WIKI/IBM', start_date=start_date, end_date=end_date, collapse='daily')

Inspect top 10 records of the retreived IBM stock market data:

In [ ]:
stock_data.head(10)

Let's also evaluate the data quality of the download by creating a set of summary statistics of the retrieved data:

In [ ]:
stock_data.describe()

Visually inspect the daily adjusted closing prices of the "International Business Machines" (IBM) stock market data:

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig = plt.figure()
ax = fig.add_subplot(111)

# plot reconstruction error scatter plot
ax.plot(stock_data.index, stock_data['Adj. Close'], color='#9b59b6')

for tick in ax.get_xticklabels():
    tick.set_rotation(45)

# set axis labels
ax.set_xlabel('[time]', fontsize=10)
ax.set_xlim([pd.to_datetime('01-01-2000'), pd.to_datetime('31-12-2017')])
ax.set_ylabel('[stock adj. closing price]', fontsize=10)
ax.set_ylim(50, 220)

# set plot title
plt.title('International Business Machines (IBM) - Daily Adjusted Historical Stock Closing Prices', fontsize=10)

Pls. note that we plotted the "adjusted" daily closing prices of the IBM stock. The stock prices are adjusted by the quandl team by several types of regular corporate actions, e.g., stock dividends, stock splits. For further details on the applied adjustments pls. refer to the following reference: https://blog.quandl.com/guide-to-stock-price-calculation

Save the obtained and validated stock market data to the local data directory:

In [ ]:
# save retrieved data to local data directory
stock_data.to_csv('data/ibm_data_2010_2017_daily.csv', sep=';', encoding='utf-8')

### 2. Data Pre-Processing

In this section, we will obtain daily returns of the retrieved daily adjusted closing prices. Also, we will convert the time-series of daily returns into a set of sequences $s$ of $n$ time steps respectively. The created sequences will then be used to learn a model using an LSTM based neural network.

#### 2.1. Daily Returns Calculation

Determine the daily returns of the "International Business Machines" (IBM) daily adjusted closing prices:

In [ ]:
stock_data['RETURN'] = stock_data['Adj. Close'].pct_change()

Visually inspect the obtained daily returns:

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig = plt.figure()
ax = fig.add_subplot(111)

# plot reconstruction error scatter plot
ax.plot(stock_data.index, stock_data['RETURN'], color='#9b59b6')

for tick in ax.get_xticklabels():
    tick.set_rotation(45)

# set axis labels and limits
ax.set_xlabel('[time]', fontsize=10)
ax.set_xlim([pd.to_datetime('01-01-2000'), pd.to_datetime('31-12-2017')])
ax.set_ylabel('[daily stock returns]', fontsize=10)

# set plot title
plt.title('International Business Machines (IBM) - Daily Historical Stock Closing Prices', fontsize=10)

#### 2.2 Transform Time-Series Into Sequences

In the following we determine the number of return time-steps $n$ each individual sequence $s^{i}$ should be comprised of. Each sequence is thereby determined by the number of predictor time-steps $T$ and the prediction horizon $t+1$.

<img align="center" style="max-width: 500px" src="timesteps.png">

In this example, we will set $T$=4. This indicates that the input sequence of each sample is a vector of the previous 4 daily stock returns. The choice of $T$=4 is arbitrary and should be selected through experimentation. Furthermore, we set the prediction horizon to 1 which specifies that we aim to forecast a single future time-step.

In [ ]:
time_steps = 4 # number of predictor timesteps
horizon = 1 # number of timesteps to be predicted
sequence_length = time_steps + horizon # determine sequence length

Remove the initial return of the return time-series which is usually not applicable and therefore 'nan':

In [ ]:
stock_daily_returns = stock_data['RETURN'][1:len(stock_data['RETURN'])]

Extract individual time sequences of length $n$ from the obtained daily returns:

In [ ]:
# iterate over distinct normalized closing prices
for i in range(0, stock_daily_returns.shape[0] - sequence_length):

    # extract normalized closing price sequence 
    single_stock_sequence_data = stock_daily_returns[i:i + sequence_length].T

    # case: initial sequence
    if i == 0:

        # convert to numpy array and collect sequence of normalized closing prices
        stock_sequence_data = np.array(single_stock_sequence_data)

    # case: non-initial sequence
    else:

        # convert to numpy array and collect sequence of normalized closing prices
        stock_sequence_data  = np.vstack((stock_sequence_data , np.array(single_stock_sequence_data)))

Inspect the top five collected normalized daily closing prices sequences:

In [ ]:
stock_sequence_data[0:5,]

#### 2.3 Prepare Sequences for Neural Network Training

Set split fraction to split return sequences into training (in-sample) and validation (out-of-sample) sequences:

In [ ]:
split_fraction = 0.9
split_row = int(stock_sequence_data.shape[0] * split_fraction)

Split obtained return sequences into training (in-sample) sequences $s^{i}_{train}$ and validation (out-of-sample) sequences $s^{i}_{valid}$:

In [ ]:
train_sequences = stock_sequence_data[:split_row,]
valid_sequences = stock_sequence_data[split_row:,]

Visually inspect the obtained train and validation sequences:

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig = plt.figure()
ax = fig.add_subplot(111)

# plot daily stock returns
ax.plot(stock_data.index[:split_row,], train_sequences[:,0], c='C0', label='train')
ax.plot(stock_data.index[split_row+6:,], valid_sequences[:,0], c='C1', label='valid')

# rotate x-labels 45 degree angle
for tick in ax.get_xticklabels():
    tick.set_rotation(45)

# set axis labels and limits
ax.set_xlabel('[time]', fontsize=10)
ax.set_xlim([pd.to_datetime('01-01-2000'), pd.to_datetime('31-12-2017')])
ax.set_ylabel('[daily stock returns]', fontsize=10)

# set plot legend
plt.legend(loc="lower right", numpoints=1, fancybox=True)

# set plot title
plt.title('International Business Machines (IBM) - Daily Historical Stock Returns', fontsize=10)

Determine count (shape) of adjusted daily return train sequences $s^{i}_{train}$:

In [ ]:
train_sequences.shape

Determine count (shape) of adjusted daily return train sequences $s^{i}_{valid}$:

In [ ]:
valid_sequences.shape

Seperate each training sequence $s^{i}$ into time-steps of input returns denoted by $s^{i}_{train, input}=\{r_{t-n-1}, ..., r_{t-1}, r_{t}\}$ and the time-step of the to be predicted target return denoted by $s^{i}_{train, target}=r_{t+1}$.

<img align="center" style="max-width: 700px" src="sequencesplit.png">

In addition, we convert both the input returns as well as the target returns to PyTorch tensors:

In [ ]:
train_sequences_input = torch.from_numpy(train_sequences[:, :-1]).float()
train_sequences_target = torch.from_numpy(train_sequences[:, 1:]).float()

Let's check if the `use_gpu` flag is set. If so, we will push our train sequence tensors onto the GPUs:

In [ ]:
# case: gpu computing enabled
if use_gpu == True:
  
  # push training sequences onto the the GPUs
  train_sequences_input = train_sequences_input.cuda()
  train_sequences_target = train_sequences_target.cuda()

Seperate each validation sequence $s^{i}$ into time-steps of input returns denoted by $s^{i}_{valid, input}=\{r_{t-n-1}, ..., r_{t-1}, r_{t}\}$ and the time-step of the to be predicted target return denoted by $s^{i}_{valid, target}=r_{t+1}$. In addition, we convert both the input returns as well as the target returns to PyTorch tensors:

In [ ]:
valid_sequences_input = torch.from_numpy(valid_sequences[:, :-1]).float()
valid_sequences_target = torch.from_numpy(valid_sequences[:, 1:]).float()

Let's check if the `use_gpu` flag is set. If so, we will push our validation sequence tensors onto the GPUs: 

In [ ]:
# case: gpu computing enabled
if use_gpu == True:
  
  # push validation sequences to the GPU
  valid_sequences_input = valid_sequences_input.cuda()
  valid_sequences_target = valid_sequences_target.cuda()

To train an LSTM neural network, we tailor the dataset class provided by the PyTorch library. We overwrite the individual functions of the dataset class. So that our dataset will supply the neural network with the individual training sequences $s^{i}_{train, input}$ and corresponding targets $s^{i}_{train, target}$ throughout the training process:

In [ ]:
# define daily returns dataset
class DailyReturnsDataset(data.Dataset):

    # define the class constructor
    def __init__(self, sequences, targets):

        # init sequences and corresponding targets
        self.sequences = sequences
        self.targets = targets

    # define the length method 
    def __len__(self):

        # returns the number of samples
        return len(self.targets)

    # define the get item method
    def __getitem__(self, index):

        # determine single sequence and corresponding target
        sequence = self.sequences[index, :]
        target = self.targets[index, :]

        # return sequences and target
        return sequence, target

Once we have specified the daily returns dataset class we instantiate it using the new daily closing dataset using the prepared training input sequences $s^{i}_{train, input}$ and corresponding targets $s^{i}_{train, target}$: 

In [ ]:
train_dataset = DailyReturnsDataset(train_sequences_input, train_sequences_target)

Let's see how it works by getting the 10th sequence and its corresponding targets:

In [ ]:
train_dataset.__getitem__(10)

### 3. Neural Network Implementation and Loss Function

In this section, we will implement the LSTM architecture of the to be learned time series model. Furthermore, we will specify the loss-function, learning-rate and optimization technique used in the network training.

#### 3.1. Implementation of the LSTM Architecture

In this section we will implement the architecture of the LSTM neural network utilized to predict future returns of financial time series data, e.g. as in this example the future returns of a given stock. The neural network, which we name **'LSTM_NN'** consists in total of three layers. The first two layers correspond to LSTM cells while the third layer corresponds to a fully-connected linear layer.

The general LSTM cell structure as well as the formal definition of its individual gate functions are shown in the following (not considering the bias of each layer for simplicity):

<img align="center" style="max-width: 700px" src="lstmcell.png">

(Source: https://pytorch.org/docs/stable/nn.html)

Each LSTM layer consits of a LSTM cell exhibiting a hidden state of 51 dimensions. The third linear squeezes the 51 hidden state dimensions of the second LSTM cell into a single output dimension. The single output signal of the linear layer refers to the return of the next time-step predicted by the neural network. The choice of the implemented architecture is arbitrary and should in a real-world scenario be evaluated and selected thoroughly through experimentation.

In [ ]:
# implement the LSTMNet network architecture
class LSTMNet(nn.Module):

    # define class constructor
    def __init__(self):

        super(LSTMNet, self).__init__()

        # define lstm nn architecture
        
        # init the lstm hidden layers
        self.lstm1 = nn.LSTMCell(1, 51)  # first lstm layer
        self.lstm2 = nn.LSTMCell(51, 51)  # second lstm layer
        
        # init the linear output layer
        self.linear = nn.Linear(51, 1)  # final linear layer

    # define network forward pass
    def forward(self, input):

        # init predictions
        predictions = []

        # init the lstm hidden states
        h_t = torch.zeros(input.size(0), 51, dtype=torch.float)
        h_t2 = torch.zeros(input.size(0), 51, dtype=torch.float)

        # init the lstm cell states
        c_t = torch.zeros(input.size(0), 51, dtype=torch.float)
        c_t2 = torch.zeros(input.size(0), 51, dtype=torch.float)
        
        # case gpu computing enabled
        if use_gpu == True:
        
          # push the initialized hidden state tensors to gpu
          h_t = h_t.cuda()
          h_t2 = h_t2.cuda()

          # push the initialized hidden cell state tensors to gpu
          c_t = c_t.cuda()
          c_t2 = c_t2.cuda()
        
        # iterate over distinct time steps
        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):

            # propagate through time step data
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            prediction = self.linear(h_t2)
            
            # collect predictions
            predictions += [prediction]

        # stack predictions
        predictions = torch.stack(predictions, 1).squeeze(2)

        # return predictions
        return predictions

Now, that we have implemented our first LSTM neural network we are ready to instantiate a model to be trained:

In [ ]:
lstm_model = LSTMNet()

Let's check if the `use_gpu` flag is set. If so, we will push the entire model instance onto the GPUs:

In [ ]:
lstm_model = LSTMNet()

# case: gpu computing enabled
if use_gpu == True:
  
  # push model to the GPU
  lstm_model = lstm_model.cuda()

Once the model is initialized, we can visualize the model structure and review the implemented network architecture by execution of the following cell:

In [ ]:
# print the initialized architectures
print('[LOG] MNISTNet architecture:\n\n{}\n'.format(lstm_model))

Looks like intended? Great!

#### 3.2. Definition of the Training Loss Function and Learning Rate

We are now good to train the network. However, prior to starting the training, we need to define an appropriate loss function. Remember, we aim to train our model to learn a set of model parameters $\theta$ that minimize the prediction error of the true return $r_{t+1}$ and the by the model predicted return $\hat{r}_{t+1}$ at a given time-step $t+1$ of sequence $s^{i}$. In other words, for a given sequence of historic returns we aim to learn a function $f_\theta$ that is capable to predicts the return of the next timestep as faithfully as possible, as expressed by:

<center> $\hat{r}_{t+1} = f_\theta(r_{t}, r_{t-1}, ..., r_{t-n})$. </center>

Thereby, the training objective is to learn a set of optimal model parameters $\theta^*$ that optimize $\min_{\theta} \|r_{t+1} - f_\theta(r_{t}, r_{t-1}, ..., r_{t-n})\|$ over all time-steps $t$ contained in the set of training sequences $s_{train}$. To achieve this optimization objective, one typically minimizes a loss function $\mathcal{L_{\theta}}$ while training the neural network. In this lab we use the **'Mean Squared Error (MSE)'** loss, as denoted by:

<center> $\mathcal{L}^{MSE}_{\theta} (r_{t+1}, \hat{r}_{t+1}) = \frac{1}{N} \sum_{i=1}^N \| r_{t+1} - \hat{r}_{t+1}\|^{2}$, </center>

In [ ]:
loss_function = nn.MSELoss()

Let's check if the `use_gpu` flag is set. If so, we will also push the loss calculation onto the GPUs:

In [ ]:
# case: gpu computing enabled
if use_gpu == True:
  
  # push the loss calculation to the GPU
  loss_function = loss_function.cuda()

Throughout the training process, the PyTorch library will automatically calculate the loss magnitude, compute the gradient, and update the parameters $\theta$ of the LSTM neural network. We will use the **"Adaptive Moment Estimation Optimization" (ADAM)** technique to optimize the network parameters. Furthermore, we specify a constant learning rate of $l = 1e-06$. For each training step, the optimizer will update the model parameters $\theta$ values according to the degree of prediction error (the MSE loss).

In [ ]:
learning_rate = 1e-06 # set constant learning rate
optimizer = optim.Adam(lstm_model.parameters(), lr=learning_rate) # define optimization technique

Now that we have successfully implemented and defined the three ANN building blocks let's take some time to review the `LSTMNet` model definition as well as the `MSE loss` function. Please, read the above code and comments carefully and don't hesitate to let us know any questions you might have.

### 4. Training the Neural Network Model

In this section, we will train the LSTM neural network model (as implemented in the section above) using the prepared dataset of daily return sequences. Therefore, we will have a detailed look into the distinct training steps and monitor the training progress.

#### 4.1. Preparing the Network Training

Let's now start to learn a model by training the NN for **2,000 epochs** in mini-batches of the size of **128  sequences** per batch. This implies that the whole dataset will be fed to the network **200 times** in chunks of 128 sequences yielding to **32 mini-batches** (4'068 training sequences / 128 sequences per mini-batch) per epoch:

In [ ]:
# specify the training parameters
num_epochs = 200 # number of training epochs
mini_batch_size = 128 # size of the mini-batches

Furthermore, lets specify and instantiate a corresponding PyTorch data loader that feeds the image tensors to our neural network:

In [ ]:
dl = dataloader.DataLoader(train_dataset, batch_size=mini_batch_size, shuffle=True)

#### 4.2. Running the Network Training

Finally, we start training the model. The training procedure of each mini-batch is performed as follows: 

>1. do a forward pass through the LSTMNet network, 
>2. compute the mean-squared prediction error $\mathcal{L}^{MSE}_{\theta} (r_{t+1}, \hat{r}_{t+1}) = \frac{1}{N} \sum_{i=1}^N \| r_{t+1} - \hat{r}_{t+1}\|^{2}$, 
>3. do a backward pass through the LSTMNet network, and 
>4. update the parameters of the network $f_\theta(\cdot)$.

To ensure learning while training the LSTM model we will monitor whether the loss decreases with progressing training. Therefore, we obtain and evaluate the mean prediction performance over all mini-batches in each training epoch. Based on this evaluation we can conclude on the training progress and whether the loss is converging (indicating that the model might not improve any further).

The following elements of the network training code below should be given particular attention:
 
>- `loss.backward()` computes the gradients based on the magnitude of the reconstruction loss,
>- `optimizer.step()` updates the network parameters based on the gradient.

In [ ]:
# init collection of training epoch losses
train_epoch_losses = []

# set the model in training mode
lstm_model.train()

# init the best loss
best_loss = 100.00

# iterate over epochs
for epoch in range(0, num_epochs):

    # init collection of mini-batch losses
    train_mini_batch_losses = []
            
    # iterate over mini-batches
    for sequence_batch, target_batch in dl:

        # predict sequence output
        prediction_batch = lstm_model(sequence_batch)

        # calculate batch loss
        batch_loss = loss_function(prediction_batch, target_batch)

        # run backward gradient calculation
        batch_loss.backward()

        # update network parameters
        optimizer.step()
        
        # collect mini-batch loss
        train_mini_batch_losses.append(batch_loss.data.item())
            
    # determine mean min-batch loss of epoch
    train_epoch_loss = np.mean(train_mini_batch_losses)
    
    # print epoch loss
    now = dt.datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
    print('[LOG {}] epoch: {} train-loss: {}'.format(str(now), str(epoch), str(train_epoch_loss)))
    
    # determine mean min-batch loss of epoch
    train_epoch_losses.append(train_epoch_loss)
        
    # print epoch and save models
    if epoch % 10 == 0 and epoch > 0:
        
        # case: new best model trained
        if train_epoch_loss < best_loss:
                        
            # store new best model
            model_name = 'awsome_lstm_model_{}.pth'.format(str(epoch))
            torch.save(lstm_model.state_dict(), os.path.join("./models", model_name))
            
            # update best loss
            best_loss = train_epoch_loss
            
            # print epoch loss
            now = dt.datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
            print('[LOG {}] epoch: {} new best train-loss: {} found'.format(str(now), str(epoch), str(train_epoch_loss)))


Upon successful training let's visualize and inspect the training loss per epoch:

In [ ]:
# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

# add grid
ax.grid(linestyle='dotted')

# plot the training epochs vs. the epochs' prediction error
ax.plot(np.array(range(1, len(train_epoch_losses)+1)), train_epoch_losses, label='epoch loss (blue)')

# add axis legends
ax.set_xlabel("[training epoch $e_i$]", fontsize=10)
ax.set_ylabel("[Prediction Error $\mathcal{L}^{MSE}$]", fontsize=10)

# set plot legend
plt.legend(loc="upper right", numpoints=1, fancybox=True)

# add plot title
plt.title('Training Epochs $e_i$ vs. Prediction Error $L^{MSE}$', fontsize=10);

Ok, fantastic. The training error is nicely going down. We could train the network a couple more epochs until the error converges. But let's stay with the 2'000 training epochs for now and continue with evaluating our trained model.

### 5. Evaluation of the Trained Neural Network Model

In this section, we will conduct a visual comparison of the predicted daily returns to the actual ('true') daily returns. The comparison will encompass the daily returns of the in-sample time period as well as the returns of the out-of-sample time period. 

We will conduct the evaluation on the **CPUs** instead of the available **GPU**. This is good practice especially in instances of large datasets. Since the available RAM on a GPU is limited. Therefore, we can't upload the entire dataset on a GPU to conduct the evaluation (of course, one could also think of a batch-wise evaluation). 

#### 5.1. In-Sample Evaluation of the Trained Neural Network Model

Before evaluating our model, let's load the best performing model or an already pre-trained model (as done below). Remember, that we stored a snapshot of the model after each training epoch to our local model directory. We will now load one of the (hopefully well-performing) snapshots saved. 

In [ ]:
# init the pre-trained model architecture
lstm_model_pretrained = LSTMNet()

# set the pre-trained model name we aim to load
model_name_pretrained = 'awsome_lstm_model_12250.pth'

# restore pretrained model checkpoint
github_model_path = 'https://raw.githubusercontent.com/financial-data-science/CFDS-4/master/lab_08/models/'
lstm_model_name_pretrained = 'awsome_lstm_model_12250.pth'

# Read stored model from the remote location
lstm_bytes = urllib.request.urlopen(os.path.join(github_model_path, lstm_model_name_pretrained))

# Load tensor from io.BytesIO object
lstm_buffer = io.BytesIO(lstm_bytes.read())

# load trained models
lstm_model_pretrained.load_state_dict(torch.load(lstm_buffer, map_location='cpu'));

Let's inspect if the model was loaded successfully: 

In [ ]:
# set model in evaluation mode
lstm_model_pretrained.eval()

Determine predictions of the in-sample population of sequences:

In [ ]:
# don't calculate gradients
with torch.no_grad():

    # predict sequence output
    predictions = lstm_model_pretrained(train_sequences_input.detach().cpu())

    # collect prediction batch results
    predictions_list = predictions.detach().cpu() # detach from CUDNN to the CPU
    predictions_list = predictions_list.numpy()[:, -1].tolist() # convert to list of sequences

    # collect target batch results
    targets_list = train_sequences_target.cpu() # detach from CUDNN to the CPU
    targets_list = target_list.numpy()[:, -1].tolist() # convert to list of sequences

Plot the lstm neural network in-sample predictions vs. the actual "ground-truth" adjusted daily returns:

In [ ]:
# plot the prediction results
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [15, 5]

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(pd.to_datetime(stock_data.index).values[0:train_sequences.shape[0],], targets_list, color='C1', label='groundtruth (green)')
ax.plot(pd.to_datetime(stock_data.index).values[0:train_sequences.shape[0],], predictions_list, color='C0', label='predictions (blue)')

# set y-axis limits
ax.set_xlim(pd.to_datetime(stock_data.index).values[0], pd.to_datetime(stock_data.index).values[train_sequences.shape[0]])

# set plot legend
plt.legend(loc="lower right", numpoints=1, fancybox=True)

plt.title('LSTM NN In-Sample Prediction vs. Ground-Truth Market Prices', fontsize=10)
plt.xlabel('[time]', fontsize=8)
plt.ylabel('[market price]', fontsize=8)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)

#### 5.2. Out-of-Sample Evaluation of the Trained Neural Network Model

Determine predictions of the out-of-sample population of sequences:

In [ ]:
# don't calculate gradients
with torch.no_grad():

    # predict sequence output
    predictions = lstm_model_pretrained(valid_sequences_input.detach().cpu())

    # collect prediction batch results
    predictions_list = predictions.detach().cpu() # detach from CUDNN to the CPU
    predictions_list = predictions_list.numpy()[:, -1].tolist() # convert to list of sequences

    # collect target batch results
    targets_list = valid_sequences_target.cpu() # detach from CUDNN to the CPU
    targets_list = targets_list.numpy()[:, -1].tolist() # convert to list of sequences

Plot the lstm neural network out-of-sample predictions vs. the actual "ground-truth" adjusted daily returns:

In [ ]:
# plot the prediction results
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [15, 5]

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(pd.to_datetime(stock_data.index).values[train_sequences.shape[0]:train_sequences.shape[0]+valid_sequences.shape[0],], targets_list, color='C1', label='groundtruth (green)')
ax.plot(pd.to_datetime(stock_data.index).values[train_sequences.shape[0]:train_sequences.shape[0]+valid_sequences.shape[0],], predictions_list, color='C0', label='predictions (blue)')

# set y-axis limits
ax.set_xlim(pd.to_datetime(stock_data.index).values[train_sequences.shape[0]], pd.to_datetime(stock_data.index).values[train_sequences.shape[0]+valid_sequences.shape[0]])
#ax.set_ylim(0.0, 1.0)

# set plot legend
plt.legend(loc="lower right", numpoints=1, fancybox=True)

plt.title('LSTM NN Out-of-Sample Prediction vs. Ground-Truth Market Prices', fontsize=10)
plt.xlabel('[time]', fontsize=8)
plt.ylabel('[market price]', fontsize=8)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)

### 6. Backtest of the Trained Neural Network Model

In this section, we will backtest the python `bt` library. Python `bt` is a flexible, backtest framework that can be used to test quantitative trading strategies. In general, backtesting is the process of testing a strategy over a given data set (more details about the `bt` library can be found via: https://pmorissette.github.io/bt/). 

In order to test the predictions derived from the LSTM model we will view its predictions as trade signals. Thereby, we will interpret any positive future return prediction $r_{t+1} > 0.0$ of a sequence $s$ as a "long" (buy) signal. Likewise, we will interpret any negative future return prediction $r_{t+1} < 0.0$ of a sequence $s$ as a "short" (sell) signal.

#### 6.1. LSTM Trading Signal Preparation

Let's start by converting the out-of-sample model predictions into a trading signal as described above. Therefore, we first convert the obtained predictions into a Pandas data frame:

In [ ]:
stock_daily_predictions = pd.DataFrame(predictions_list, columns=['PREDICTIONS'])
stock_daily_predictions = stock_daily_predictions.set_index(pd.to_datetime(stock_data.index).values[train_sequences.shape[0]:train_sequences.shape[0]+valid_sequences.shape[0],])

Furthermore, let's briefly ensure the successful conversion by inspecting the top 5 rows of the predictions:

In [ ]:
stock_daily_predictions.head(10)

Now, let's derive a trading signal from the converted predictions. As already described, we will generate the trading signal $\phi$ according to the following function:

<center>
$
\\
\phi(\hat{r}_{t+1})=
\begin{cases}
1.0, & for & \hat{r}_{t+1} > 0.0\\
-1.0, & for & \hat{r}_{t+1} < 0.0\\
\end{cases}
$
</center>

where $\hat{r}_{t+1}$ denotes a by the model predicted future return at time $t+1$.

In [ ]:
signal_data = pd.DataFrame(np.where(stock_daily_predictions['PREDICTIONS'] > 0.0, 1.0, -1.0), columns=['SIGNAL'])
signal_data = signal_data.set_index(pd.to_datetime(stock_data.index).values[train_sequences.shape[0]:train_sequences.shape[0]+valid_sequences.shape[0],])

Let's inspect the top 5 rows of the prepared trading signals:

In [ ]:
signal_data.head(5)

#### 6.2. Stock Market Data Preperation

Now, let's prepare the daily adjusted closing prices so that they can be utilized in the backtest:

In [ ]:
stock_market_data = pd.DataFrame(stock_data['Adj. Close'])
stock_market_data = stock_market_data.rename(columns={'Adj. Close': 'PRICE'})
stock_market_data = stock_market_data.set_index(pd.to_datetime(stock_data.index))

Let's inspect the top 5 rows of the prepared adjusted closing prices:

In [ ]:
stock_market_data.head(5)

Sub-sample the prepared daily adjusted closing prices to the out-of-sample time period:

In [ ]:
stock_market_data = stock_market_data[stock_market_data.index >= stock_daily_predictions.index[0]]
stock_market_data = stock_market_data[stock_market_data.index <= stock_daily_predictions.index[-1]]

Visualize the out-of-sample daily adjusted closing prices:

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(stock_market_data['PRICE'], color='#9b59b6')

for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    
# set axis labels
ax.set_xlabel('[time]', fontsize=10)
#ax.set_xlim(pd.to_datetime(stock_market_data.index).values[train_sequences.shape[0]], pd.to_datetime(stock_data.index).values[train_sequences.shape[0]+valid_sequences.shape[0]])
ax.set_ylabel('[equity %]', fontsize=10)

for tick in ax.get_xticklabels():
    tick.set_rotation(45)

# set axis labels
ax.set_xlabel('[time]', fontsize=10)
ax.set_xlim([stock_market_data.index[0], stock_market_data.index[-1]])
ax.set_ylabel('[adj. closing price]', fontsize=10)

# set plot title
plt.title('International Business Machines Corporation (IBM) - Daily Historical Stock Closing Prices', fontsize=10)

Let's calculate our potential gained return by the application of a simple "buy and hold" strategy:

In [ ]:
np.abs(stock_market_data.iloc[0]['PRICE'] - stock_market_data.iloc[-1]['PRICE']) / stock_market_data.iloc[0]['PRICE']

#### 6.3. Backtest Preparation

Implement the "Moving Average" trading strategy:

In [ ]:
class LSTMStrategy(bt.Algo):
    
    def __init__(self, signals):
        
        # set class signals
        self.signals = signals
        
    def __call__(self, target):
        
        if target.now in self.signals.index[1:]:
            
            # get actual signal
            signal = self.signals.ix[target.now]
            
            # set target weights according to signal
            target.temp['weights'] = dict(PRICE=signal)
            
        # return True since we want to move on to the next timestep
        return True

Initialize our LSTM based trading strategy:

In [ ]:
lstm_strategy = bt.Strategy('lstm', [bt.algos.SelectAll(), LSTMStrategy(signal_data['SIGNAL']), bt.algos.Rebalance()])

Initialize the backtest of our LSTM based trading strategy using the strategy and prepared market data:

In [ ]:
backtest_lstm = bt.Backtest(strategy=lstm_strategy, data=stock_market_data, name='stock_lstm_backtest')

In addition, let's also prepare a backtest of a "baseline" buy-and-hold trading strategy for comparison purposes. Our buy-and-hold strategy sends a "long" (+1.0) signal at each time step of the out-of-sample time frame: 

In [ ]:
signal_data_base = signal_data.copy(deep=True) 
signal_data_base['SIGNAL'] = 1.0

Init the buy-and-hold ("base") strategy as well as the corresponding backtest:

In [ ]:
base_strategy = bt.Strategy('base', [bt.algos.SelectAll(), LSTMStrategy(signal_data_base['SIGNAL']), bt.algos.Rebalance()])
backtest_base = bt.Backtest(strategy=base_strategy, data=stock_market_data, name='stock_base_backtest')

#### 6.4. Running the Backtest and Evaluate Results

Run the backtest for both trading strategies: 

In [ ]:
backtest_results = bt.run(backtest_lstm, backtest_base)

Inspect the individual backtest results and performance measures:

In [ ]:
backtest_results.display()

Collect the LSTM trading strategy backtest details:

In [ ]:
backtest_lstm_details = backtest_lstm.strategy.data
backtest_lstm_details.columns = ['% EQUITY', 'EQUITY', 'CASH', 'FEES']

Inspect the LSTM trading strategy backtest details:

In [ ]:
backtest_lstm_details.head(10)

Collect the "buy-and-hold" trading strategy backtest details:

In [ ]:
backtest_base_details = backtest_base.strategy.data
backtest_base_details.columns = ['% EQUITY', 'EQUITY', 'CASH', 'FEES']

Inspect the "buy-and-hold" trading strategy backtest details:

In [ ]:
backtest_base_details.head(10)

Visualize the equity progression of both strategies over time:

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(backtest_lstm_details['% EQUITY'], color='C1',lw=1.0, label='lstm strategy (green)')
ax.plot(backtest_base_details['% EQUITY'], color='C2',lw=1.0, label='base strategy (red)')

for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    
# set axis labels
ax.set_xlabel('[time]', fontsize=10)
ax.set_xlim(pd.to_datetime(stock_data.index).values[train_sequences.shape[0]], pd.to_datetime(stock_data.index).values[train_sequences.shape[0]+valid_sequences.shape[0]])
ax.set_ylabel('[equity %]', fontsize=10)

# set plot legend
plt.legend(loc="upper right", numpoints=1, fancybox=True)

# set plot title
plt.title('International Business Machines Corporation (IBM) - Backtest % Equity Progression', fontsize=10)

### Exercises:

We recommend you to try the following exercises as part of the lab:

**1. Evaluation of Shallow vs. Deep RNN Models.**

> Download the daily adjusted closing prices of the IBM stock within the time frame starting from 01/01/1990 until 05/31/2019. In addition to the architecture of the lab notebook, evaluate further (more shallow as well as more deep) RNN architectures by (1) either re-moving/adding layers of LSTM cells and/or (2) increasing/decreasing the dimensionality of the LSTM cells hidden state. Train your model (using architectures you selected) for at least 20'000 training epochs but keep the following parameters unchanged (a) sequence length: 5 time-steps (days) and (b) train vs. test fraction: 0.9.

> Analyze the prediction performance of the trained models in terms of training time and prediction accuracy. Furthermore, backtest the out-of-sample signals predicted by each of your models and evaluate them in terms of total return and equity progression. Which of your architecture results in the best performing model, and why?

In [ ]:
# ***************************************************
# INSERT YOUR CODE HERE
# ***************************************************

**2. Training and Evaluation of Models Learned from Additional Stocks.**

> Download the daily adjusted closing prices of at least two additional stocks (e.g., Alphabet, Deutsche Bank) within the time frame starting from 01/01/1990 until 05/31/2017. Pls. select two stocks that you are interested in to investigate ( e.g. stocks that you may occasionally trade yourself). Learn an ’optimal’ RNN model of both stocks and backtest their corresponding trade signals by following the approach outlined in the lab notebook regarding the IBM stock. Pls. keep the train vs. test dataset fraction fixed to 0.9, all other parameters of the data preparation and model training can be changed.

> Analyse the performance of the learned models in terms of their prediction accuracy as well as their out-of-sample backtest performance (e. g. the total return and equity progression). What architectures and corresponding training parameters result in the best performing models?

In [ ]:
# ***************************************************
# INSERT YOUR CODE HERE
# ***************************************************

**3. Training and Evaluation of Models Learned from Augmented Data.**

> In the prior exercises, we used the historical daily adjusted returns of a single stock to learn a model that can predict the stocks’ future adjusted closing price (log- return) movement. However, one of the advantages of NN’s lies in their capability to learn a model from multiple sources of input data.
For each of the two stocks (’target stocks’) that you selected in exercise 2. learn an ’optimal’ RNN model using the daily returns as a target label. However, before training your models augment the training data of each stock by the return sequences of at least three additional stocks. The additional stocks, used for data augmentation, should exhibit a high correlation to the historical adjusted closing prices of the target stock price movement you aim to model.

> Analyse the performance of the learned models in terms of their prediction accuracy as well as their out-of-sample backtest performance (e. g. the total return and equity progression). Do you observe an improvement of the trained model in terms of out-of-sample backtest performance comparison to exercise 1.?

In [ ]:
# ***************************************************
# INSERT YOUR CODE HERE
# ***************************************************

### Lab Summary:

In this seventh lab, a step by step introduction into **design, implementation, training and evaluation** of a LSTM neural network based trading strategy is presented. 

The strategy trades a specific financial instrument based on its historical adjusted daily market prices. The degree of success of the implemented strategy is evaluated based in its backtest performance with particular focus on (1) the strategy's **total return** as well as (2) its **equity progression** over time. 

The code provided in this lab provides a blueprint for the development and testing of more complex trading strategies.

You may want to execute the content of your lab outside of the Jupyter notebook environment, e.g. on a compute node or a server. The cell below converts the lab notebook into a standalone and executable python script. Pls. note that to convert the notebook, you need to install Python's **nbconvert** library and its extensions:

In [ ]:
# installing the nbconvert library
!pip install nbconvert
!pip install jupyter_contrib_nbextensions

Let's now convert the Jupyter notebook into a plain Python script:

In [ ]:
!jupyter nbconvert --to script cfds_lab_08.ipynb